In [4]:
import random
import json
import re
import os
from pathlib import Path

import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix

import nltk
import nltk
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer as wnl

import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm
random.seed(6)
np.random.seed(6)

WORK_DIR = Path.cwd().parent


In [3]:
# Download required NLTK data
# Run if needed
# nltk.download('punkt_tab')
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('wordnet')


In [6]:
def get_vocab_map(label_dir):
    vocab_map = dict()

    with open(label_dir) as in_file:
        for line in in_file.readlines():
            mapping = line.strip('\n')
            key, val = mapping.split(' ')
            vocab_map[key] = val
    
    return vocab_map

def textual_stsg(q_data):
    frame_ids = sorted(q_data['situations'].keys())

    stsg = {}
    for f in frame_ids:
        frame_sg = []
        rels = q_data['situations'][f]['rel_labels']
        
        if rels == []:
            # There are some frames in the ground truth without any scene graph
            # annotation
            continue
        for rel_pair, rel in zip(q_data['situations'][f]['rel_pairs'], rels):
            obj1, obj2 = rel_pair
            frame_sg.append([obj_vocab[obj1], rel_vocab[rel], obj_vocab[obj2]])

        stsg[f] = frame_sg
    
    return stsg


obj_vocab = get_vocab_map(WORK_DIR / 'data/datasets/STAR/STAR_annotations/class_maps/object_classes.txt')
rel_vocab = get_vocab_map(WORK_DIR /  'data/datasets/STAR/STAR_annotations/class_maps/relationship_classes.txt')


In [7]:
obj_vocab


{'o000': 'person',
 'o001': 'broom',
 'o002': 'picture',
 'o003': 'closet/cabinet',
 'o004': 'blanket',
 'o005': 'window',
 'o006': 'table',
 'o007': 'paper/notebook',
 'o008': 'refrigerator',
 'o009': 'pillow',
 'o010': 'cup/glass/bottle',
 'o011': 'shelf',
 'o012': 'shoe',
 'o013': 'medicine',
 'o014': 'phone/camera',
 'o015': 'box',
 'o016': 'sandwich',
 'o017': 'book',
 'o018': 'bed',
 'o019': 'clothes',
 'o020': 'mirror',
 'o021': 'sofa/couch',
 'o022': 'floor',
 'o023': 'bag',
 'o024': 'dish',
 'o025': 'laptop',
 'o026': 'door',
 'o027': 'towel',
 'o028': 'food',
 'o029': 'chair',
 'o030': 'doorknob',
 'o031': 'doorway',
 'o032': 'groceries',
 'o033': 'hands',
 'o034': 'light',
 'o035': 'vacuum',
 'o036': 'television'}

In [8]:
rel_vocab


{'r000': 'on',
 'r001': 'behind',
 'r002': 'in_front_of',
 'r003': 'on_the_side_of',
 'r004': 'above',
 'r005': 'beneath',
 'r006': 'drinking_from',
 'r007': 'have_it_on_the_back',
 'r008': 'wearing',
 'r009': 'holding',
 'r010': 'lying_on',
 'r011': 'covered_by',
 'r012': 'carrying',
 'r013': 'eating',
 'r014': 'leaning_on',
 'r015': 'sitting_on',
 'r016': 'twisting',
 'r017': 'writing_on',
 'r018': 'standing_on',
 'r019': 'touching',
 'r020': 'wiping',
 'r021': 'at',
 'r022': 'under',
 'r023': 'near'}

### Wordent
Let's use `wordent` an try to aggregate words with simialry meaning.


> WordNet® is a large lexical database of English. Nouns, verbs, adjectives and adverbs are grouped into sets of cognitive synonyms (synsets), each expressing a distinct concept. Synsets are interlinked by means of conceptual-semantic and lexical relations. 
> 
> WordNet superficially resembles a thesaurus, in that it groups words together based on their meanings. However, there are some important distinctions. 
> - First, WordNet interlinks not just word forms—strings of letters—but specific senses of words. As a result, words that are found in close proximity to one another in the network are semantically disambiguated. 
> - Second, WordNet labels the semantic relations among words, whereas the groupings of words in a thesaurus does not follow any explicit pattern other than meaning similarity.

In [5]:
# get the different synsets (concept) that can be associated to
# the word in the specified part-of-speech (pos)
# 
# Synset: a set of synonyms that share a common meaning.
wn.synsets('man', pos=wn.NOUN)


[Synset('man.n.01'),
 Synset('serviceman.n.01'),
 Synset('man.n.03'),
 Synset('homo.n.02'),
 Synset('man.n.05'),
 Synset('man.n.06'),
 Synset('valet.n.01'),
 Synset('man.n.08'),
 Synset('man.n.09'),
 Synset('man.n.10'),
 Synset('world.n.08')]